# Distribution of Police Stations in Metropolitan Cities of India

In [3]:
import numpy as np 

import pandas as pd
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
import requests 
from pandas.io.json import json_normalize 
!conda install -c conda-forge folium=0.5.0 --yes 

print('Libraries imported.')

Solving environment: done

## Package Plan ##

  environment location: /opt/conda/envs/Python36

  added / updated specs: 
    - folium=0.5.0


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    ca-certificates-2020.6.20  |       hecda079_0         145 KB  conda-forge
    branca-0.4.1               |             py_0          26 KB  conda-forge
    python_abi-3.6             |          1_cp36m           4 KB  conda-forge
    vincent-0.4.4              |             py_1          28 KB  conda-forge
    certifi-2020.6.20          |   py36h9f0ad1d_0         151 KB  conda-forge
    folium-0.5.0               |             py_0          45 KB  conda-forge
    openssl-1.1.1g             |       h516909a_0         2.1 MB  conda-forge
    altair-4.1.0               |             py_1         614 KB  conda-forge
    ------------------------------------------------------------
                       

In [15]:
import folium
CLIENT_ID = 'Y23RDWDL3QTMGDCQWR5BXOT3BUZS5GV2UFYX3GUG3QQUGNWX'
CLIENT_SECRET = 'W5OJJR2WR0F05VTAF3NRJ0U1QPYKUUH4KZWXTRXKUTRN2ASS' 
VERSION = '20180605' 

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)

Your credentails:
CLIENT_ID: Y23RDWDL3QTMGDCQWR5BXOT3BUZS5GV2UFYX3GUG3QQUGNWX


In [19]:
LIMIT = 500 # Maximum is 100
cities = ["New Delhi, National Capital Territory of Delhi, India", 'Bangalore, Karnātaka, India', 'Mumbai, Mahārāshtra','Lucknow, Uttar Pradesh, India']
results = {}
for city in cities:
    url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&near={}&limit={}&categoryId={}'.format(
        CLIENT_ID, 
        CLIENT_SECRET, 
        VERSION, 
        city,
        LIMIT,
        "4bf58dd8d48988d12e941735") #Foursquare ID Of Police Station
    results[city] = requests.get(url).json()

In [20]:
df_venues={}
for city in cities:
    venues = json_normalize(results[city]['response']['groups'][0]['items'])
    df_venues[city] = venues[['venue.name', 'venue.location.address', 'venue.location.lat', 'venue.location.lng']]
    df_venues[city].columns = ['Name', 'Address', 'Lat', 'Lng']

### We observe the distribution of Police Stations in each city.

In [21]:
maps = {}
for city in cities:
    city_lat = np.mean([results[city]['response']['geocode']['geometry']['bounds']['ne']['lat'],
                        results[city]['response']['geocode']['geometry']['bounds']['sw']['lat']])
    city_lng = np.mean([results[city]['response']['geocode']['geometry']['bounds']['ne']['lng'],
                        results[city]['response']['geocode']['geometry']['bounds']['sw']['lng']])
    maps[city] = folium.Map(location=[city_lat, city_lng], zoom_start=11)

    # add markers to map
    for lat, lng, label in zip(df_venues[city]['Lat'], df_venues[city]['Lng'], df_venues[city]['Name']):
        label = folium.Popup(label, parse_html=True)
        folium.CircleMarker(
            [lat, lng],
            radius=50,
            popup=label,
            color='blue',
            fill=True,
            fill_color='#3186cc',
            fill_opacity=0.7,
            parse_html=False).add_to(maps[city])  
    print(f"Total number of police stations in {city} = ", results[city]['response']['totalResults'])
    print("Showing Top 100")

Total number of police stations in New Delhi, National Capital Territory of Delhi, India =  4
Showing Top 100
Total number of police stations in Bangalore, Karnātaka, India =  17
Showing Top 100
Total number of police stations in Mumbai, Mahārāshtra =  46
Showing Top 100
Total number of police stations in Lucknow, Uttar Pradesh, India =  6
Showing Top 100


#### New Delhi

In [22]:
maps[cities[0]]

#### Bangalore

In [23]:
maps[cities[1]]

#### Mumbai

In [24]:
maps[cities[2]]

#### Lucknow

In [25]:
maps[cities[3]]

#### We see that even though number of police stations in Mumbai is more, the density is greater in Bangalore.

#### Next we will get the mean location of the Police Stations which should be near to most of them if they are really dense or far if not.

#### Then we will take the average of the distance of the venues to the mean coordinates.

In [26]:
maps = {}
for city in cities:
    city_lat = np.mean([results[city]['response']['geocode']['geometry']['bounds']['ne']['lat'],
                        results[city]['response']['geocode']['geometry']['bounds']['sw']['lat']])
    city_lng = np.mean([results[city]['response']['geocode']['geometry']['bounds']['ne']['lng'],
                        results[city]['response']['geocode']['geometry']['bounds']['sw']['lng']])
    maps[city] = folium.Map(location=[city_lat, city_lng], zoom_start=11)
    venues_mean_coor = [df_venues[city]['Lat'].mean(), df_venues[city]['Lng'].mean()] 
    # add markers to map
    for lat, lng, label in zip(df_venues[city]['Lat'], df_venues[city]['Lng'], df_venues[city]['Name']):
        label = folium.Popup(label, parse_html=True)
        folium.CircleMarker(
            [lat, lng],
            radius=5,
            popup=label,
            color='blue',
            fill=True,
            fill_color='#3186cc',
            fill_opacity=0.7,
            parse_html=False).add_to(maps[city])
        folium.PolyLine([venues_mean_coor, [lat, lng]], color="green", weight=1.5, opacity=0.5).add_to(maps[city])
    
    label = folium.Popup("Mean Co-ordinate", parse_html=True)
    folium.CircleMarker(
        venues_mean_coor,
        radius=10,
        popup=label,
        color='green',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(maps[city])

    print(city)
    print("Mean Distance from Mean coordinates")
    print(np.mean(np.apply_along_axis(lambda x: np.linalg.norm(x - venues_mean_coor),1,df_venues[city][['Lat','Lng']].values)))

New Delhi, National Capital Territory of Delhi, India
Mean Distance from Mean coordinates
0.05473523082961468
Bangalore, Karnātaka, India
Mean Distance from Mean coordinates
0.05809142047400903
Mumbai, Mahārāshtra
Mean Distance from Mean coordinates
0.09155085017257628
Lucknow, Uttar Pradesh, India
Mean Distance from Mean coordinates
0.01558841116226777


#### New Delhi

In [27]:
maps[cities[0]]

#### Bangalore

In [28]:
maps[cities[1]]

#### Mumbai

In [29]:
maps[cities[2]]

#### Lucknow

In [30]:
maps[cities[3]]

### Conclusion : The network of police stations is best connected in Bangalore.